In [4]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from flask import Flask, request, render_template
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from threading import Thread

# Load the dataset
data = pd.read_csv('dataset.txt')

# Initialize Flask app
app = Flask(__name__)

# Define routes
@app.route('/')
def home():
    return render_template('ProjectHomepage.html')

@app.route('/soilfertility', methods=['POST'])
def soilfertility():
    # Collect input data
    inputs = [float(request.form[str(i)]) for i in range(14)]

    if all(x == 0 for x in inputs):
        return render_template('ProjectHomepage.html', prediction_text=0)

    # Prepare the dataset and input for prediction
    X, Y = data[data.columns[1:]], data['Vegetation Cover']
    dict_input = {col_name: [value] for col_name, value in zip(X.columns, inputs)}
    df_input = pd.DataFrame(dict_input)
    
    # Concatenate input with the dataset and normalize
    df = pd.concat([X, df_input], ignore_index=True)
    scaler = MinMaxScaler()
    X_scaled, Y_scaled = scaler.fit_transform(df.values), scaler.fit_transform(Y.values.reshape(-1,1))
    
    # Prepare the input for the model
    l1 = [X_scaled[-1]]
    X_scaled = X_scaled[:-1]  # Remove the last row used for prediction

    # Split data
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.10, random_state=43)

    # Train the model
    forestRegressor = RandomForestRegressor(criterion='mse', max_depth=8, n_estimators=10, random_state=0)
    forestRegressor.fit(X_train, Y_train)

    # Make prediction
    prediction = forestRegressor.predict(l1)
    prediction = prediction[0]

    # Create response text
    text = "Your soil is high fertile." if prediction >= 0.9 else "Your Soil is less fertile."
    if prediction < 0.9:
        nutrients = []
        if inputs[0] < 12.75:
            nutrients.append("NO3")
        if inputs[2] < 47:
            nutrients.append("P")
        if inputs[8] < 0.6:
            nutrients.append("Zn")
        if inputs[3] < 15:
            nutrients.append("K")
        if inputs[6] < 0.28:
            nutrients.append("Organic Matter")
        if inputs[10] < 1:
            nutrients.append("Fe")
        if nutrients:
            text += " Consider increasing these nutrients: " + ", ".join(nutrients)
    
    return render_template('Results.html', content=text, prediction_text=int(np.round(prediction * 100)))

# Run Flask in a separate thread to keep Jupyter responsive
def run_flask():
    app.run(port=8000, debug=False, use_reloader=False)

if __name__ == "__main__":
    thread = Thread(target=run_flask)
    thread.start()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Aug/2024 19:45:11] "GET / HTTP/1.1" 200 -
C:\Users\ranaw\AppData\Local\Temp\ipykernel_18184\3152527160.py:50: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forestRegressor.fit(X_train, Y_train)
C:\Users\ranaw\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
127.0.0.1 - - [09/Aug/2024 19:45:20] "POST /soilfertility HTTP/1.1" 200 -
